In [2]:
import cv2
import numpy as np
import json
from shapely.geometry import LineString, mapping
import geojson
import os


# List of annotated image paths
image_paths = [
    "dounambay/annotations/dounambay2005_annotated.png",
    "dounambay/annotations/dounambayapril2013_annotated.png",
    "dounambay/annotations/dounambayapril2019_annotated.png",
    "dounambay/annotations/dounambayapril2024_annotated.png",
    "dounambay/annotations/dounambayjuly2011_annotated.png",
    "dounambay/annotations/dounambaymay2019_annotated.png",
    "dounambay/annotations/dounambaymay2021_annotated.png",
]

# Prepare to store GeoJSON features
features = []

# Process each image
for image_path in image_paths:
    # Extract the file name without the extension
    image_name = os.path.basename(image_path).replace('.png', '')
    
    # Load the image
    image = cv2.imread(image_path)
    
    # Check if the image is loaded correctly
    if image is None:
        print(f"Error loading image: {image_path}")
        continue
    
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply threshold to create a binary image
    _, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    
    # Detect edges using Canny
    edges = cv2.Canny(binary, 100, 200)
    
    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Create a black image
    coastline_image = np.zeros_like(gray)
    
    # Draw the contours in white
    cv2.drawContours(coastline_image, contours, -1, (255, 255, 255), 1)
    
    # Save the contour image
    coastline_image_path = f"dounambay/coastline/{image_name}_coastline.png"
    cv2.imwrite(coastline_image_path, coastline_image)
    
    # Convert the contours to GeoJSON
    for contour in contours:
        if len(contour) > 1:  # Ensure the contour has more than one point
            coordinates = contour[:, 0, :].tolist()
            linestring = LineString(coordinates)
            feature = {
                "type": "Feature",
                "geometry": mapping(linestring),
                "properties": {
                    "image": image_name  # File name without extension
                }
            }
            features.append(feature)

# Create a GeoJSON FeatureCollection
geojson_dict = {
    "type": "FeatureCollection",
    "features": features
}

# Save the GeoJSON data)
print(os.listdir())
geojson_file_path = "coastlines.json"
with open(geojson_file_path, 'w') as f:
    geojson.dump(geojson_dict, f)

# Display the paths to the saved images and the GeoJSON file
coastline_image_paths = [f"dounambay/coastline/{os.path.basename(image_path).replace('.png', '')}_coastline.png" for image_path in image_paths]
print(coastline_image_paths, geojson_file_path)


['autolabeler.old.ipynb', 'autolabeler4.ipynb', 'autolabeler4.py', 'autolabelerv2.ipynb', 'autolabelerv3.ipynb', 'autolabelling.py', 'best_classifier_model.keras', 'best_model.keras', 'best_segmentation_model.keras', 'classification', 'classification.ipynb', 'classification.py', 'classification.zip', 'classification2.zip', 'coastlines.json', 'coastline_classifier.h5', 'coastline_classifier.keras', 'coastline_classifier_1.keras', 'coastline_classifier_10.keras', 'coastline_classifier_11.keras', 'coastline_classifier_12.keras', 'coastline_classifier_13.keras', 'coastline_classifier_2.keras', 'coastline_classifier_3.keras', 'coastline_classifier_4.keras', 'coastline_classifier_5.keras', 'coastline_classifier_6.keras', 'coastline_classifier_7.keras', 'coastline_classifier_8.keras', 'coastline_classifier_9.keras', 'coastline_classifier_epoch_10.keras', 'coastline_classifier_epoch_100.keras', 'coastline_classifier_epoch_101.keras', 'coastline_classifier_epoch_102.keras', 'coastline_classifie